In [34]:
from jetbot import Camera, bgr8_to_jpeg
import traitlets
import ipywidgets as widgets
from IPython.display import display
import cv2
import numpy as np
import math
from io import BytesIO
import time
from jetbot import Robot

#設定照片大小
photo_width = 240
photo_height = 240
offset = 50

# 获取图像尺寸和 ROI
center_x = photo_width // 2
center_y = int(photo_height * 0.6)
w = h = 10
x = center_x - w // 2
y = center_y - h // 2

#左側ROI
left_x = center_x // 2 - w
left_y = center_y -offset
left_w = 20
left_h = 10

#右側ROI
right_x = center_x + center_x // 2
right_y = center_y -offset
right_w = 20
right_h = 10

# 创建显示图像的小工具
image_widget = widgets.Image(format='jpeg', width=photo_width, height=photo_height)
mask_widget = widgets.Image(format='jpeg', width=photo_width, height=photo_height)
red_mask_widget = widgets.Image(format='jpeg', width=photo_width, height=photo_height)  # 新增红色遮罩显示
hsv_label = widgets.Label(value="ROI Average HSV: H=0, S=0, V=0")
speed_label = widgets.Label(value="Speed: L=0, R=0")
obstacle_label = widgets.Label(value="Obstacle Status: No")  # 新增障碍物状态显示

# 创建线条追踪的 HSV 范围滑动条
h_lower = widgets.IntSlider(min=0, max=179, value=20, description='H Lower')
h_upper = widgets.IntSlider(min=0, max=179, value=90, description='H Upper')
s_lower = widgets.IntSlider(min=0, max=255, value=0, description='S Lower')
s_upper = widgets.IntSlider(min=0, max=255, value=255, description='S Upper')
v_lower = widgets.IntSlider(min=0, max=255, value=140, description='V Lower')
v_upper = widgets.IntSlider(min=0, max=255, value=255, description='V Upper')

# 创建红色检测的 HSV 范围滑动条
red_h_lower = widgets.IntSlider(min=0, max=179, value=140, description='Red H Lower')
red_h_upper = widgets.IntSlider(min=0, max=179, value=0, description='Red H Upper')
red_s_lower = widgets.IntSlider(min=0, max=255, value=110, description='Red S Lower')
red_s_upper = widgets.IntSlider(min=0, max=255, value=240, description='Red S Upper')
red_v_lower = widgets.IntSlider(min=0, max=255, value=75, description='Red V Lower')
red_v_upper = widgets.IntSlider(min=0, max=255, value=255, description='Red V Upper')

# 创建追踪控制滑动条和开关
search_radius_slider = widgets.IntSlider(value=300, min=50, max=400, step=10, description='Search Radius')
tracking_toggle = widgets.ToggleButton(value=False, description='Enable Tracking', button_style='success')

# 水平排列三个图像窗口
image_box = widgets.HBox([image_widget, mask_widget, red_mask_widget])

# 垂直排列控制组件
control_box = widgets.VBox([
    hsv_label,
    speed_label,
    obstacle_label,
    h_lower, h_upper,
    s_lower, s_upper,
    v_lower, v_upper,
    widgets.HTML(value="<b>Red Detection Parameters:</b>"),
    red_h_lower, red_h_upper,
    red_s_lower, red_s_upper,
    red_v_lower, red_v_upper,
    search_radius_slider,
    tracking_toggle
])

# 显示所有控件
display(image_box, control_box)

# 初始化
camera = Camera.instance(width=photo_width, height=photo_height)
robot = Robot()

# 避障状态
avoiding_obstacle = False
avoid_start_time = 0
AVOID_DURATION = 3.0  # 避障持续时间（秒）

def find_nearest_white(mask, center_x, center_y, search_radius, step_angle=5, step_radius=10):
    # 初始化變量
    min_distance = float('inf')
    max_area = 0
    best_point = None
    best_contour = None
    
    # 找到所有的輪廓
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 如果沒有找到任何輪廓，返回None
    if not contours:
        return None
    
    # 遍歷每個輪廓
    for contour in contours:
        # 計算輪廓的面積
        area = cv2.contourArea(contour)
        
        # 如果面積太小，跳過
        if area < 50:  # 可以調整這個閾值
            continue
            
        # 找到輪廓的質心
        M = cv2.moments(contour)
        if M["m00"] == 0:
            continue
            
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        
        # 確保點在搜索範圍內
        if cy >= center_y:  # 跳過低於中心點的區域
            continue
            
        # 計算到中心點的距離
        dx = cx - center_x
        dy = cy - center_y
        distance = math.sqrt(dx*dx + dy*dy)
        
        # 如果超出搜索半徑，跳過
        if distance > search_radius:
            continue
            
        # 結合距離和面積的評分
        # 這裡我們給予面積和距離不同的權重
        # 可以調整這些權重來改變選擇的偏好
        distance_weight = 0.3
        area_weight = 0.7
        
        # 正規化距離和面積 (越小越好)
        normalized_distance = distance / search_radius
        normalized_area = 1 - (area / max(cv2.contourArea(c) for c in contours))
        
        # 計算總評分 (越小越好)
        score = distance_weight * normalized_distance + area_weight * normalized_area
        
        # 更新最佳點
        if score < min_distance or (score == min_distance and area > max_area):
            min_distance = score
            max_area = area
            best_point = (cx, cy)
            best_contour = contour
    
    # 如果找到了合適的點，在圖像上標記整個輪廓
    if best_point is not None and 'image' in globals():
        cv2.drawContours(image, [best_contour], -1, (0, 255, 0), 2)
        
    return best_point

def detect_red_obstacle(hsv):
    global w, h, x, y
    # 当red_h_lower > red_h_upper时，需要分段处理HSV的H通道
    if red_h_lower.value > red_h_upper.value:
        # 第一段：从red_h_lower到179
        lower_red1 = np.array([red_h_lower.value, red_s_lower.value, red_v_lower.value])
        upper_red1 = np.array([179, red_s_upper.value, red_v_upper.value])
        
        # 第二段：从0到red_h_upper
        lower_red2 = np.array([0, red_s_lower.value, red_v_lower.value])
        upper_red2 = np.array([red_h_upper.value, red_s_upper.value, red_v_upper.value])
    else:
        # 正常情况：单一区间
        lower_red1 = np.array([red_h_lower.value, red_s_lower.value, red_v_lower.value])
        upper_red1 = np.array([red_h_upper.value, red_s_upper.value, red_v_upper.value])
        
        lower_red2 = np.array([0, 0, 0])
        upper_red2 = np.array([0, 0, 0])
    
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)
    
    red_roi_mask_mid = red_mask[y-offset:y+h-offset, x:x+w]
    red_ratio = np.sum(red_roi_mask_mid == 255) / (w * h)
    
    red_roi_mask_left = red_mask[left_y:left_y+left_h, left_x:left_x+left_w]
    red_ratio_left = np.sum(red_roi_mask_left == 255) / (left_w * left_h)
    
    red_roi_mask_right = red_mask[right_y:right_y+right_h, right_x:right_x+right_w]
    red_ratio_right = np.sum(red_roi_mask_right == 255) / (right_w * right_h)
    
    if red_ratio_left > 0.5:
        position = 1
    elif red_ratio_right > 0.5:
        position = 3
    else:
        position = 2

    has_obstacle = red_ratio >= 0.5 or red_ratio_left >= 0.5 or red_ratio_right >= 0.5
    
    return red_mask, has_obstacle, position

def process_image(image):
    global avoiding_obstacle, avoid_start_time
    global center_x, center_y, w, h, x, y
    global left_x, left_y, left_w, left_h
    global right_x, right_y, right_w, right_h
    
    # 初始化返回值
    angle = 0.0
    distance = 0.0
    l_speed = 0
    r_speed = 0
    
    # 转换到 HSV 色彩空间
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # 创建基础mask用于线追踪
    lower_val = np.array([h_lower.value, s_lower.value, v_lower.value])
    upper_val = np.array([h_upper.value, s_upper.value, v_upper.value])
    mask = cv2.inRange(hsv, lower_val, upper_val)
    mask_color = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    # 检测红色障碍物
    red_mask, has_obstacle, position = detect_red_obstacle(hsv)
    red_mask_color = cv2.cvtColor(red_mask, cv2.COLOR_GRAY2BGR)
    
    # 更新障碍物状态显示
    if has_obstacle and not avoiding_obstacle:
        obstacle_label.value = "Obstacle Status: YES"
        avoiding_obstacle = True
        if (time.time() - avoid_start_time) > 10:
            avoid_start_time = time.time()
    elif avoiding_obstacle and (time.time() - avoid_start_time) > AVOID_DURATION:
        avoiding_obstacle = False
        obstacle_label.value = "Obstacle Status: No"
    
    # ROI的HSV平均值计算
    roi_hsv = hsv[y:y+h, x:x+w]
    mean_hsv = np.mean(roi_hsv, axis=(0,1)).astype(int)
    hsv_label.value = f"ROI Average HSV: H={mean_hsv[0]}, S={mean_hsv[1]}, V={mean_hsv[2]}"
    
    # 绘制所有ROI矩形
    # 中心ROI
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 1)
    cv2.rectangle(mask_color, (x, y), (x+w, y+h), (255, 0, 0), 1)
    cv2.rectangle(red_mask_color, (x, y-offset), (x+w, y+h-offset), (255, 0, 0), 1)
    
    # 左侧ROI
    cv2.rectangle(image, (left_x, left_y), (left_x+left_w, left_y+left_h), (0, 255, 0), 1)
    cv2.rectangle(mask_color, (left_x, left_y), (left_x+left_w, left_y+left_h), (0, 255, 0), 1)
    cv2.rectangle(red_mask_color, (left_x, left_y), (left_x+left_w, left_y+left_h), (0, 255, 0), 1)
    
    # 右侧ROI
    cv2.rectangle(image, (right_x, right_y), (right_x+right_w, right_y+right_h), (0, 0, 255), 1)
    cv2.rectangle(mask_color, (right_x, right_y), (right_x+right_w, right_y+right_h), (0, 0, 255), 1)
    cv2.rectangle(red_mask_color, (right_x, right_y), (right_x+right_w, right_y+right_h), (0, 0, 255), 1)
    
    if avoiding_obstacle:
        # 如果正在避障，返回特定的转向指令
        l_speed = 0
        r_speed = 0
        if (time.time() - avoid_start_time) < 0.05:
            if position == 1:
                robot.set_motors(0.15, -0.15)
                time.sleep(0.2)
                robot.set_motors(0.3, 0.3)
                time.sleep(0.4)
                robot.stop()

                avoiding_obstacle = False
            elif position == 3:
                robot.set_motors(-0.15, 0.15)
                time.sleep(0.2)
                robot.set_motors(0.3, 0.3)
                time.sleep(0.4)
                robot.stop()

                avoiding_obstacle = False
            elif position == 2:
                robot.set_motors(-0.15, 0.15)
                time.sleep(0.4)
                robot.stop()
                robot.set_motors(0.3, 0.3)
                time.sleep(0.5)
                robot.set_motors(0.15, -0.15)
                time.sleep(0.4)
                robot.set_motors(0.30, 0.30)
                time.sleep(0.5)
                robot.stop()
                avoiding_obstacle = False
            else:
                robot.stop()
                avoiding_obstacle = False
                
        else:
            robot.stop()

    else:
        # 正常的线追踪逻辑
        roi_mask = mask[y:y+h, x:x+w]
        white_ratio = np.sum(roi_mask == 255) / (w * h)
        
        if white_ratio < 0.5:
            nearest = find_nearest_white(mask, center_x, center_y, search_radius_slider.value)
            
            if nearest is not None:
                for img in [image, mask_color]:
                    cv2.circle(img, nearest, 3, (0, 255, 0), -1)
                    cv2.line(img, (center_x, center_y), nearest, (0, 255, 0), 1)
                
                dx = nearest[0] - center_x
                dy = center_y - nearest[1]
                angle = -math.degrees(math.atan2(dx, dy))
                distance = math.sqrt(dx*dx + dy*dy)
                
                cv2.putText(image, f'A: {angle:.0f} D: {distance:.0f}', 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        l_speed = 0.12 + angle * -0.0009
        r_speed = 0.18 + angle * 0.0009
        
    # 返回所有图像和计算值
    return bgr8_to_jpeg(image), bgr8_to_jpeg(mask_color), bgr8_to_jpeg(red_mask_color), angle, distance, l_speed, r_speed

def update_image(change):
    image = change['new']
    image_jpeg, mask_jpeg, red_mask_jpeg, angle, distance, l_speed, r_speed = process_image(image)
    image_widget.value = image_jpeg
    mask_widget.value = mask_jpeg
    red_mask_widget.value = red_mask_jpeg
    
    speed_label.value = f"Speed: L={l_speed:.3f}, R={r_speed:.3f}"
    
    if tracking_toggle.value:
        robot.set_motors(l_speed, r_speed)
    else:
        robot.stop()

In [35]:
# 连接摄像头输出到处理函数
camera.observe(update_image, names='value')

In [36]:
camera.stop()
robot.stop()